In [1]:
using Polynomials
using QuadGK;

# M10.8
---
Comparison of interpolation and approximation in regard to the Runge function:

$$f(x)=\frac{1}{25x^2+1} \qquad x\in[-1, 1]$$

## Interpolation

In [2]:
function get_newton_coeffs(nodes, f_values)
    coeffs = Float64[]
    len = length(nodes)
    
    # First iteration
    diff_quots = collect(f_values)
    push!(coeffs, diff_quots[1])
    
    # Next iterations
    for j = 1:len-1
        for i = 1:len-j
            diff_quots[i] = ((diff_quots[i] - diff_quots[i + 1])
                             / (nodes[i] - nodes[i + j]))
        end
        push!(coeffs, diff_quots[1])
    end
    coeffs
end


function interpolate(f, nodes)::Poly
    coeffs = get_newton_coeffs(nodes, f.(nodes))
    
    # First iteration
    nodal_poly = Poly([1])
    inter_poly = Poly([coeffs[1]])
    
    # Next iterations
    for i = 2:length(coeffs)
        nodal_poly *= Poly([-nodes[i - 1], 1])
        inter_poly += coeffs[i] * nodal_poly
    end
    inter_poly
end;

In [3]:
function get_equidistant_nodes(nodes_number)
    linspace(-1, 1, nodes_number)
end


function get_chebyshev_zeros_nodes(nodes_number)
    [cos(pi * (2*k + 1) / (2*nodes_number)) for k = nodes_number-1:-1:0]
end


function get_chebyshev_extremes_nodes(nodes_number)
    [cos(pi * k / (nodes_number-1)) for k = nodes_number-1:-1:0]
end;

## Approximation

In [4]:
function approximate_legendre(f, deg)
    optimal_poly = Poly([0])
    P = Poly[]
    push!(P, Poly([1]))
    push!(P, Poly([0, 1]))
    
    for i = 0:deg
        if i >= length(P)
            push!(P, Poly([0, (2i - 1) / i])*P[end] - ((i - 1) / i)*P[end-1])
        end
        a = (2i + 1) / 2 * quadgk(x -> f(x) * polyval(P[i+1], x), -1,  1)[1]
        optimal_poly += a*P[i+1]
    end
    optimal_poly
end;

In [5]:
function approximate_chebyshev(f, deg)
    optimal_poly = Poly([0])
    T = Poly[]
    push!(T, Poly([1]))
    push!(T, Poly([0, 1]))
    
    p(x) = 1 / sqrt(1 - x^2)
    for i = 0:deg
        if i >= length(T)
            push!(T, Poly([0, 2])*T[end] - T[end-1])
        end
        a = 2 / pi * quadgk(x -> p(x) * f(x) * polyval(T[i+1], x), -0.99999, 0.99999)[1]
        if i == 0 a /= 2 end
        optimal_poly += a*T[i+1]
    end
    optimal_poly
end;

## Tests

In [7]:
uniform_norm(f, poly) = maximum(map(x -> abs(f(x) - polyval(poly, x)), linspace(-1, 1, 10000)))
f(x) = 1 / (25*x^2 + 1)

for (nodes_generator, nodes_name) in zip(
        [get_equidistant_nodes, get_chebyshev_zeros_nodes, get_chebyshev_extremes_nodes],
        ["Equidistant nodes", "Chebyshev zeros", "Chebyshev extremes"]
)
    nodes = nodes_generator(10)
    inter_poly = interpolate(f, nodes)
    @printf "%25.25s - %.10f\n" nodes_name uniform_norm(f, inter_poly)
end

for (approximation_func, approximation_name) in zip(
        [approximate_legendre, approximate_chebyshev],
        ["Legendre approximation", "Chebyshev approximation"]
)
    optimal_poly = approximation_func(f, 9)
    @printf "%25.25s - %.10f\n" approximation_name uniform_norm(f, optimal_poly)
end;

        Equidistant nodes - 0.3002977778
          Chebyshev zeros - 0.2691781334
       Chebyshev extremes - 0.3190950441
   Legendre approximation - 0.1495833235
  Chebyshev approximation - 0.1641203605
